Sklearn

In [1]:
"""Example using an sklearn Pipeline with TuneGridSearchCV.
Example taken and modified from
https://scikit-learn.org/stable/auto_examples/compose/
plot_compare_reduction.html
"""

from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2

pipe = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ("reduce_dim", "passthrough"),
    ("classify", LinearSVC(dual=False, max_iter=10000))
])

N_FEATURES_OPTIONS = [2, 4, 8]
C_OPTIONS = [1, 10]
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF()],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS
    },
    {
        "reduce_dim": [SelectKBest(chi2)],
        "reduce_dim__k": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS
    },
]

random = TuneSearchCV(pipe, param_grid, search_optimization="random")

In [6]:
X, y = load_digits(return_X_y=True)
X = X[:100]
y = y[:100]

In [7]:
random.fit(X, y)
print(random.cv_results_)

grid = TuneGridSearchCV(pipe, param_grid=param_grid)
grid.fit(X, y)
print(grid.cv_results_)

D:\anaconda3\envs\yourenvname\lib\site-packages\ray\tune\tune.py:373: UserWarning: The `loggers` argument is deprecated. Please pass the respective `LoggerCallback` classes to the `callbacks` argument instead. See https://docs.ray.io/en/latest/tune/api_docs/logging.html
  warnings.warn(
save not implemented for Searcher. Skipping save.
Log sync requires rsync to be installed.
(pid=None) [2021-12-05 20:35:21,367 C 2872 5176] redis_client.cc:87:  Check failed: under_retry_limit Expected 1 Redis shard addresses, found 2
(pid=None) *** StackTrace Information ***
(pid=None)     configthreadlocale
(pid=None)     BaseThreadInitThunk
(pid=None)     RtlUserThreadStart
(pid=None) 
 pid=16192) C:\Users\James\AppData\Roaming\Python\Python38\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
 pid=16192)   warnings.warn(
 pid=59

RayActorError: The actor died because of an error raised in its creation task, [36mray::_Trainable.__init__()[39m (pid=5984, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 625, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 578, in ray._raylet.execute_task.function_executor
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\_private\function_manager.py", line 609, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\util\tracing\tracing_helper.py", line 451, in _resume_span
    return method(self, *_args, **_kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\tune\trainable.py", line 121, in __init__
    self.setup(copy.deepcopy(self.config))
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\util\tracing\tracing_helper.py", line 451, in _resume_span
    return method(self, *_ar

Basic Example

In [8]:
"""This example demonstrates basic Ray Tune random search and grid search."""
import time

import ray
from ray import tune


def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
    if args.server_address is not None:
        ray.init(f"ray://{args.server_address}")
    else:
        ray.init(configure_logging=False)

    # This will do a grid search over the `activation` parameter. This means
    # that each of the two values (`relu` and `tanh`) will be sampled once
    # for each sample (`num_samples`). We end up with 2 * 50 = 100 samples.
    # The `width` and `height` parameters are sampled randomly.
    # `steps` is a constant parameter.

    analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        num_samples=5 if args.smoke_test else 10,
        config={
            "steps": 5 if args.smoke_test else 10,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
            "activation": tune.grid_search(["relu", "tanh"])
        })

    print("Best hyperparameters found were: ", analysis.best_config)

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,activation,height,width
easy_objective_15877_00000,RUNNING,127.0.0.1:13020,relu,-79.4372,19.7305
easy_objective_15877_00001,PENDING,,tanh,34.1997,10.509
easy_objective_15877_00002,PENDING,,relu,38.4212,2.42528
easy_objective_15877_00003,PENDING,,tanh,81.9109,1.92543
easy_objective_15877_00004,PENDING,,relu,-93.3214,12.4241
easy_objective_15877_00005,PENDING,,tanh,-96.8686,13.8366
easy_objective_15877_00006,PENDING,,relu,94.5131,8.92664
easy_objective_15877_00007,PENDING,,tanh,73.9598,16.7644
easy_objective_15877_00008,PENDING,,relu,28.2088,1.06464
easy_objective_15877_00009,PENDING,,tanh,86.737,18.8427


Result for easy_objective_15877_00000:
  date: 2021-12-05_20-36-44
  done: false
  experiment_id: bfa16cf210de4a73a5a9ba3dea1984a7
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 2.056276871923412
  neg_mean_loss: -2.056276871923412
  node_ip: 127.0.0.1
  pid: 13020
  time_since_restore: 0.10644125938415527
  time_this_iter_s: 0.10644125938415527
  time_total_s: 0.10644125938415527
  timestamp: 1638689804
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '15877_00000'
  


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_15877_00000,RUNNING,127.0.0.1:13020,relu,-79.4372,19.7305,-6.81914,5,0.773405,4,6.81914
easy_objective_15877_00001,RUNNING,127.0.0.1:3172,tanh,34.1997,10.509,,,,,
easy_objective_15877_00002,RUNNING,127.0.0.1:11260,relu,38.4212,2.42528,,,,,
easy_objective_15877_00003,RUNNING,127.0.0.1:15308,tanh,81.9109,1.92543,,,,,
easy_objective_15877_00004,PENDING,,relu,-93.3214,12.4241,,,,,
easy_objective_15877_00005,PENDING,,tanh,-96.8686,13.8366,,,,,
easy_objective_15877_00006,PENDING,,relu,94.5131,8.92664,,,,,
easy_objective_15877_00007,PENDING,,tanh,73.9598,16.7644,,,,,
easy_objective_15877_00008,PENDING,,relu,28.2088,1.06464,,,,,
easy_objective_15877_00009,PENDING,,tanh,86.737,18.8427,,,,,


Result for easy_objective_15877_00003:
  date: 2021-12-05_20-36-49
  done: false
  experiment_id: eca19c84e6f740a49cecccfe379e1b83
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 18.191094461501308
  neg_mean_loss: -18.191094461501308
  node_ip: 127.0.0.1
  pid: 15308
  time_since_restore: 0.10448646545410156
  time_this_iter_s: 0.10448646545410156
  time_total_s: 0.10448646545410156
  timestamp: 1638689809
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '15877_00003'
  


...

Result for easy_objective_15877_00016:
  date: 2021-12-05_20-37-26
  done: false
  experiment_id: d1cbf293573146e3bfb5f8cce773f01f
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.112398063890371
  neg_mean_loss: -10.112398063890371
  node_ip: 127.0.0.1
  pid: 16968
  time_since_restore: 0.10545802116394043
  time_this_iter_s: 0.10545802116394043
  time_total_s: 0.10545802116394043
  timestamp: 1638689846
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '15877_00016'
  
Result for easy_objective_15877_00017:
  date: 2021-12-05_20-37-26
  done: false
  experiment_id: 0f0f28992dfb40c3a1586b05af35ce42
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 5.519742519181848
  neg_mean_loss: -5.519742519181848
  node_ip: 127.0.0.1
  pid: 4228
  time_since_restore: 0.10155677795410156
  time_this_iter_s: 0.10155677795410156
  time_total_s: 0.10155677795410156
  timestamp: 1638689846
  timesteps_since_res

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_15877_00016,RUNNING,127.0.0.1:16968,relu,1.12398,14.0382,2.7387,3,0.440398,2,-2.7387
easy_objective_15877_00017,RUNNING,127.0.0.1:4228,tanh,-44.8026,2.79671,1.9327,3,0.336891,2,-1.9327
easy_objective_15877_00018,RUNNING,127.0.0.1:13860,relu,20.2672,10.8603,5.17925,3,0.332009,2,-5.17925
easy_objective_15877_00019,RUNNING,127.0.0.1:3564,tanh,27.5975,16.6238,5.07201,3,0.319317,2,-5.07201
easy_objective_15877_00000,TERMINATED,127.0.0.1:13020,relu,-79.4372,19.7305,-7.4106,10,6.60994,9,7.4106
easy_objective_15877_00001,TERMINATED,127.0.0.1:3172,tanh,34.1997,10.509,4.37617,10,1.43056,9,-4.37617
easy_objective_15877_00002,TERMINATED,127.0.0.1:11260,relu,38.4212,2.42528,6.98405,10,1.5624,9,-6.98405
easy_objective_15877_00003,TERMINATED,127.0.0.1:15308,tanh,81.9109,1.92543,11.8502,10,1.49795,9,-11.8502
easy_objective_15877_00004,TERMINATED,127.0.0.1:11432,relu,-93.3214,12.4241,-8.51124,10,1.34171,9,8.51124
easy_objective_15877_00005,TERMINATED,127.0.0.1:17356,tanh,-96.8686,13.8366,-8.94353,10,1.34171,9,8.94353


Result for easy_objective_15877_00017:
  date: 2021-12-05_20-37-27
  done: true
  experiment_id: 0f0f28992dfb40c3a1586b05af35ce42
  experiment_tag: 17_activation=tanh,height=-44.803,width=2.7967
  hostname: LAPTOP-5F3V9FNM
  iterations: 9
  iterations_since_restore: 10
  mean_loss: -1.6369566106425295
  neg_mean_loss: 1.6369566106425295
  node_ip: 127.0.0.1
  pid: 4228
  time_since_restore: 1.0868468284606934
  time_this_iter_s: 0.10936975479125977
  time_total_s: 1.0868468284606934
  timestamp: 1638689847
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: '15877_00017'
  
Result for easy_objective_15877_00016:
  date: 2021-12-05_20-37-27
  done: true
  experiment_id: d1cbf293573146e3bfb5f8cce773f01f
  experiment_tag: 16_activation=relu,height=1.124,width=14.038
  hostname: LAPTOP-5F3V9FNM
  iterations: 9
  iterations_since_restore: 10
  mean_loss: 0.8458382943734882
  neg_mean_loss: -0.8458382943734882
  node_ip: 127.0.0.1
  pid: 16968
  time_since_restore: 1.2040181159

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_15877_00000,TERMINATED,127.0.0.1:13020,relu,-79.4372,19.7305,-7.4106,10,6.60994,9,7.4106
easy_objective_15877_00001,TERMINATED,127.0.0.1:3172,tanh,34.1997,10.509,4.37617,10,1.43056,9,-4.37617
easy_objective_15877_00002,TERMINATED,127.0.0.1:11260,relu,38.4212,2.42528,6.98405,10,1.5624,9,-6.98405
easy_objective_15877_00003,TERMINATED,127.0.0.1:15308,tanh,81.9109,1.92543,11.8502,10,1.49795,9,-11.8502
easy_objective_15877_00004,TERMINATED,127.0.0.1:11432,relu,-93.3214,12.4241,-8.51124,10,1.34171,9,8.51124
easy_objective_15877_00005,TERMINATED,127.0.0.1:17356,tanh,-96.8686,13.8366,-8.94353,10,1.34171,9,8.94353
easy_objective_15877_00006,TERMINATED,127.0.0.1:8384,relu,94.5131,8.92664,10.5582,10,1.26164,9,-10.5582
easy_objective_15877_00007,TERMINATED,127.0.0.1:5264,tanh,73.9598,16.7644,8.01757,10,1.36026,9,-8.01757
easy_objective_15877_00008,TERMINATED,127.0.0.1:14976,relu,28.2088,1.06464,7.92768,10,5.45764,9,-7.92768
easy_objective_15877_00009,TERMINATED,127.0.0.1:1680,tanh,86.737,18.8427,9.23054,10,5.52797,9,-9.23054


Best hyperparameters found were:  {'steps': 10, 'width': 13.836608953363786, 'height': -96.86863456065052, 'activation': 'tanh'}


Async Hyperband

In [10]:
import argparse
import time

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler


def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be an arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--ray-address",
        help="Address of Ray cluster for seamless distributed execution.",
        required=False)
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
#     if args.server_address is not None:
#         ray.init(f"ray://{args.server_address}")
#     else:
#         ray.init(address=args.ray_address)

    # AsyncHyperBand enables aggressive early stopping of bad trials.
    scheduler = AsyncHyperBandScheduler(grace_period=5, max_t=100)

    # 'training_iteration' is incremented every time `trainable.step` is called
    stopping_criteria = {"training_iteration": 1 if args.smoke_test else 9999}

    analysis = tune.run(
        easy_objective,
        name="asynchyperband_test",
        metric="mean_loss",
        mode="min",
        scheduler=scheduler,
        stop=stopping_criteria,
        num_samples=3,
        verbose=1,
        resources_per_trial={
            "cpu": 1,
            "gpu": 0
        },
        config={  # Hyperparameter space
            "steps": 5,
            "width": tune.uniform(10, 100),
            "height": tune.uniform(0, 100),
        })
    print("Best hyperparameters found were: ", analysis.best_config)

2021-12-05 20:39:18,090	INFO tune.py:626 -- Total run time: 16.12 seconds (15.99 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 5, 'width': 79.75077163085182, 'height': 11.773148399995092}


Hyperband

In [12]:
import argparse
import json
import os

import numpy as np

import ray
from ray import tune
from ray.tune.schedulers import HyperBandScheduler


def train(config, checkpoint_dir=None):
    step = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            step = json.loads(f.read())["timestep"]

    for timestep in range(step, 100):
        v = np.tanh(float(timestep) / config.get("width", 1))
        v *= config.get("height", 1)

        # Checkpoint the state of the training every 3 steps
        # Note that this is only required for certain schedulers
        if timestep % 3 == 0:
            with tune.checkpoint_dir(step=timestep) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                with open(path, "w") as f:
                    f.write(json.dumps({"timestep": timestep}))

        # Here we use `episode_reward_mean`, but you can also report other
        # objectives such as loss or accuracy.
        tune.report(episode_reward_mean=v)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
#     if args.server_address is not None:
#         ray.init(f"ray://{args.server_address}")
#     else:
#         ray.init(num_cpus=4 if args.smoke_test else None)

    # Hyperband early stopping, configured with `episode_reward_mean` as the
    # objective and `training_iteration` as the time unit,
    # which is automatically filled by Tune.
    hyperband = HyperBandScheduler(max_t=200)

    analysis = tune.run(
        train,
        name="hyperband_test",
        num_samples=3,
        metric="episode_reward_mean",
        mode="max",
        stop={"training_iteration": 3 if args.smoke_test else 4},
        config={"height": tune.uniform(0, 100)},
        scheduler=hyperband,
        fail_fast=True)
    print("Best hyperparameters found were: ", analysis.best_config)

2021-12-05 20:41:15,909	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,height
train_ba36c_00000,PENDING,,9.21705
train_ba36c_00001,PENDING,,26.1167
train_ba36c_00002,PENDING,,1.04658


Trial name,status,loc,height
train_ba36c_00000,RUNNING,127.0.0.1:12404,9.21705
train_ba36c_00001,PENDING,,26.1167
train_ba36c_00002,PENDING,,1.04658


Result for train_ba36c_00000:
  date: 2021-12-05_20-41-23
  done: false
  episode_reward_mean: 0.0
  experiment_id: c8ec24e2f68c43b8bbaefd6e4694dde9
  hostname: LAPTOP-5F3V9FNM
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 12404
  should_checkpoint: true
  time_since_restore: 0.006841182708740234
  time_this_iter_s: 0.006841182708740234
  time_total_s: 0.006841182708740234
  timestamp: 1638690083
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ba36c_00000
  
Result for train_ba36c_00000:
  date: 2021-12-05_20-41-23
  done: true
  episode_reward_mean: 9.17146632252167
  experiment_id: c8ec24e2f68c43b8bbaefd6e4694dde9
  hostname: LAPTOP-5F3V9FNM
  iterations_since_restore: 4
  node_ip: 127.0.0.1
  pid: 12404
  should_checkpoint: true
  time_since_restore: 0.032225847244262695
  time_this_iter_s: 0.006836652755737305
  time_total_s: 0.032225847244262695
  timestamp: 1638690083
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: ba36c_00000
  


Trial name,status,loc,height,iter,total time (s),reward
train_ba36c_00001,RUNNING,127.0.0.1:8148,26.1167,,,
train_ba36c_00002,PENDING,,1.04658,,,
train_ba36c_00000,TERMINATED,127.0.0.1:12404,9.21705,4,0.0322258,9.17147


...

Trial name,status,loc,height,iter,total time (s),reward
train_ba36c_00002,RUNNING,127.0.0.1:14908,1.04658,,,
train_ba36c_00000,TERMINATED,127.0.0.1:12404,9.21705,4,0.0322258,9.17147
train_ba36c_00001,TERMINATED,127.0.0.1:8148,26.1167,4,0.0800478,25.9875


Result for train_ba36c_00002:
  date: 2021-12-05_20-41-32
  done: false
  episode_reward_mean: 0.0
  experiment_id: c619578c020d4e9193f52720e18b0c2c
  hostname: LAPTOP-5F3V9FNM
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 14908
  should_checkpoint: true
  time_since_restore: 0.002933502197265625
  time_this_iter_s: 0.002933502197265625
  time_total_s: 0.002933502197265625
  timestamp: 1638690092
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ba36c_00002
  
Result for train_ba36c_00002:
  date: 2021-12-05_20-41-32
  done: true
  episode_reward_mean: 1.0414084143356162
  experiment_id: c619578c020d4e9193f52720e18b0c2c
  hostname: LAPTOP-5F3V9FNM
  iterations_since_restore: 4
  node_ip: 127.0.0.1
  pid: 14908
  should_checkpoint: true
  time_since_restore: 0.03808712959289551
  time_this_iter_s: 0.016604900360107422
  time_total_s: 0.03808712959289551
  timestamp: 1638690092
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: ba36c_00002
  


Trial name,status,loc,height,iter,total time (s),reward
train_ba36c_00000,TERMINATED,127.0.0.1:12404,9.21705,4,0.0322258,9.17147
train_ba36c_00001,TERMINATED,127.0.0.1:8148,26.1167,4,0.0800478,25.9875
train_ba36c_00002,TERMINATED,127.0.0.1:14908,1.04658,4,0.0380871,1.04141


2021-12-05 20:41:32,624	INFO tune.py:626 -- Total run time: 16.74 seconds (16.57 seconds for the tuning loop).


Best hyperparameters found were:  {'height': 26.11670014580333}


PopulationBasedTraining

In [13]:

import numpy as np
import argparse
import json
import os
import random

import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining


def pbt_function(config, checkpoint_dir=None):
    """Toy PBT problem for benchmarking adaptive learning rate.

    The goal is to optimize this trainable's accuracy. The accuracy increases
    fastest at the optimal lr, which is a function of the current accuracy.

    The optimal lr schedule for this problem is the triangle wave as follows.
    Note that many lr schedules for real models also follow this shape:

     best lr
      ^
      |    /\
      |   /  \
      |  /    \
      | /      \
      ------------> accuracy

    In this problem, using PBT with a population of 2-4 is sufficient to
    roughly approximate this lr schedule. Higher population sizes will yield
    faster convergence. Training will not converge without PBT.
    """
    lr = config["lr"]
    accuracy = 0.0  # end = 1000
    start = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            accuracy = state["acc"]
            start = state["step"]

    midpoint = 100  # lr starts decreasing after acc > midpoint
    q_tolerance = 3  # penalize exceeding lr by more than this multiple
    noise_level = 2  # add gaussian noise to the acc increase
    # triangle wave:
    #  - start at 0.001 @ t=0,
    #  - peak at 0.01 @ t=midpoint,
    #  - end at 0.001 @ t=midpoint * 2,
    for step in range(start, 100):
        if accuracy < midpoint:
            optimal_lr = 0.01 * accuracy / midpoint
        else:
            optimal_lr = 0.01 - 0.01 * (accuracy - midpoint) / midpoint
        optimal_lr = min(0.01, max(0.001, optimal_lr))

        # compute accuracy increase
        q_err = max(lr, optimal_lr) / min(lr, optimal_lr)
        if q_err < q_tolerance:
            accuracy += (1.0 / q_err) * random.random()
        elif lr > optimal_lr:
            accuracy -= (q_err - q_tolerance) * random.random()
        accuracy += noise_level * np.random.normal()
        accuracy = max(0, accuracy)

        if step % 3 == 0:
            with tune.checkpoint_dir(step=step) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                with open(path, "w") as f:
                    f.write(json.dumps({"acc": accuracy, "step": start}))

        tune.report(
            mean_accuracy=accuracy,
            cur_lr=lr,
            optimal_lr=optimal_lr,  # for debugging
            q_err=q_err,  # for debugging
            done=accuracy > midpoint * 2  # this stops the training process
        )


def run_tune_pbt():
    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=4,
        hyperparam_mutations={
            # distribution for resampling
            "lr": lambda: random.uniform(0.0001, 0.02),
            # allow perturbations within this set of categorical values
            "some_other_factor": [1, 2],
        })

    analysis = tune.run(
        pbt_function,
        name="pbt_test",
        scheduler=pbt,
        verbose=False,
        metric="mean_accuracy",
        mode="max",
        stop={
            "training_iteration": 3,
        },
        num_samples=3,
        fail_fast=True,
        config={
            "lr": 0.0001,
            # note: this parameter is perturbed but has no effect on
            # the model training in this example
            "some_other_factor": 1,
        })

    print("Best hyperparameters found were: ", analysis.best_config)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
#     if args.smoke_test:
#         ray.init(num_cpus=2)  # force pausing to happen for test
#     else:
#         if args.server_address is not None:
#             ray.init(f"ray://{args.server_address}")
#         else:
#             ray.init()

    run_tune_pbt()

2021-12-05 20:42:49,704	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Best hyperparameters found were:  {'lr': 0.0001, 'some_other_factor': 1}


Logging

In [14]:
import argparse
import time

from ray import tune
from ray.tune.logger import LoggerCallback


class TestLoggerCallback(LoggerCallback):
    def on_trial_result(self, iteration, trials, trial, result, **info):
        print(f"TestLogger for trial {trial}: {result}")


def trial_str_creator(trial):
    return "{}_{}_123".format(trial.trainable_name, trial.trial_id)


def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()

#     if args.server_address:
#         import ray
#         ray.init(f"ray://{args.server_address}")

    analysis = tune.run(
        easy_objective,
        name="hyperband_test",
        metric="mean_loss",
        mode="min",
        num_samples=3,
        trial_name_creator=trial_str_creator,
        callbacks=[TestLoggerCallback()],
        stop={"training_iteration": 1 if args.smoke_test else 5},
        config={
            "steps": 100,
            "width": tune.randint(10, 100),
            "height": tune.loguniform(10, 100)
        })
    print("Best hyperparameters: ", analysis.best_config)

2021-12-05 20:43:53,642	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,height,width
easy_objective_183b9_00000_123,PENDING,,18.9115,50
easy_objective_183b9_00001_123,PENDING,,75.3468,16
easy_objective_183b9_00002_123,PENDING,,33.8706,15


Trial name,status,loc,height,width
easy_objective_183b9_00000_123,PENDING,,18.9115,50
easy_objective_183b9_00001_123,PENDING,,75.3468,16
easy_objective_183b9_00002_123,PENDING,,33.8706,15


Trial name,status,loc,height,width
easy_objective_183b9_00000_123,RUNNING,127.0.0.1:14220,18.9115,50
easy_objective_183b9_00001_123,RUNNING,127.0.0.1:17168,75.3468,16
easy_objective_183b9_00002_123,PENDING,,33.8706,15


TestLogger for trial easy_objective_183b9_00001_123: {'iterations': 0, 'mean_loss': 17.5346810586663, 'time_this_iter_s': 0.10155391693115234, 'done': False, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 1, 'neg_mean_loss': -17.5346810586663, 'trial_id': '183b9_00001', 'experiment_id': '4cb533ccf86e4c33b559ed0433c60ffc', 'date': '2021-12-05_20-44-03', 'timestamp': 1638690243, 'time_total_s': 0.10155391693115234, 'pid': 17168, 'hostname': 'LAPTOP-5F3V9FNM', 'node_ip': '127.0.0.1', 'config': {'steps': 100, 'width': 16, 'height': 75.346810586663}, 'time_since_restore': 0.10155391693115234, 'timesteps_since_restore': 0, 'iterations_since_restore': 1}
Result for easy_objective_183b9_00001_123:
  date: 2021-12-05_20-44-03
  done: false
  experiment_id: 4cb533ccf86e4c33b559ed0433c60ffc
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 17.5346810586663
  neg_mean_loss: -17.5346810586663
  node_ip: 127.0.0.1
  pid: 17168
  time_since

...

Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_183b9_00002_123,RUNNING,127.0.0.1:1860,33.8706,15,13.3871,1,0.117182,0,-13.3871
easy_objective_183b9_00000_123,TERMINATED,127.0.0.1:14220,18.9115,50,2.36734,5,0.540003,4,-2.36734
easy_objective_183b9_00001_123,TERMINATED,127.0.0.1:17168,75.3468,16,8.88603,5,0.529263,4,-8.88603


TestLogger for trial easy_objective_183b9_00002_123: {'iterations': 1, 'mean_loss': 7.387063247077819, 'time_this_iter_s': 0.1503772735595703, 'done': False, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 2, 'neg_mean_loss': -7.387063247077819, 'trial_id': '183b9_00002', 'experiment_id': '0b95a0a0d87b46f1a19fc6f8918a32a9', 'date': '2021-12-05_20-44-09', 'timestamp': 1638690249, 'time_total_s': 0.2675597667694092, 'pid': 1860, 'hostname': 'LAPTOP-5F3V9FNM', 'node_ip': '127.0.0.1', 'config': {'steps': 100, 'width': 15, 'height': 33.87063247077819}, 'time_since_restore': 0.2675597667694092, 'timesteps_since_restore': 0, 'iterations_since_restore': 2}
TestLogger for trial easy_objective_183b9_00002_123: {'iterations': 2, 'mean_loss': 5.887063247077819, 'time_this_iter_s': 0.11624789237976074, 'done': False, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 3, 'neg_mean_loss': -5.887063247077819, 'trial_id': '183b9_00002', 'experiment_id': '0b95a

Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_183b9_00000_123,TERMINATED,127.0.0.1:14220,18.9115,50,2.36734,5,0.540003,4,-2.36734
easy_objective_183b9_00001_123,TERMINATED,127.0.0.1:17168,75.3468,16,8.88603,5,0.529263,4,-8.88603
easy_objective_183b9_00002_123,TERMINATED,127.0.0.1:1860,33.8706,15,4.81563,5,0.60543,4,-4.81563


2021-12-05 20:44:09,598	INFO tune.py:626 -- Total run time: 15.97 seconds (15.83 seconds for the tuning loop).


Best hyperparameters:  {'steps': 100, 'width': 50, 'height': 18.911529382486783}


hyperopt_example

In [6]:
"""This example demonstrates the usage of HyperOpt with Ray Tune.

It also checks that it is usable with a separate scheduler.
"""
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch


def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()

#     if args.server_address:
#         ray.init(f"ray://{args.server_address}")
#     else:
#         ray.init(configure_logging=False)

    current_best_params = [
        {
            "width": 1,
            "height": 2,
            "activation": "relu"  # Activation will be relu
        },
        {
            "width": 4,
            "height": 2,
            "activation": "tanh"  # Activation will be tanh
        }
    ]

    algo = HyperOptSearch(points_to_evaluate=current_best_params)
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    scheduler = AsyncHyperBandScheduler()
    analysis = tune.run(
        easy_objective,
        search_alg=algo,
        scheduler=scheduler,
        metric="mean_loss",
        mode="min",
        num_samples=2 if args.smoke_test else 3,
        config={
            "steps": 3,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
            # This is an ignored parameter.
            "activation": tune.choice(["relu", "tanh"])
        })

    print("Best hyperparameters found were: ", analysis.best_config)

2021-12-05 21:07:15,885	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,activation,height,steps,width
easy_objective_5c09f8a3,PENDING,,relu,2,3,1


Result for easy_objective_5c09f8a3:
  date: 2021-12-05_21-07-19
  done: false
  experiment_id: cc4ea00f6bf54ade874ad7efd24fe7a2
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 127.0.0.1
  pid: 2912
  time_since_restore: 0.1015481948852539
  time_this_iter_s: 0.1015481948852539
  time_total_s: 0.1015481948852539
  timestamp: 1638691639
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5c09f8a3
  
Result for easy_objective_5c09f8a3:
  date: 2021-12-05_21-07-20
  done: true
  experiment_id: cc4ea00f6bf54ade874ad7efd24fe7a2
  experiment_tag: 1_activation=relu,height=2.0,steps=3,width=1.0
  hostname: LAPTOP-5F3V9FNM
  iterations: 2
  iterations_since_restore: 3
  mean_loss: 8.533333333333331
  neg_mean_loss: -8.533333333333331
  node_ip: 127.0.0.1
  pid: 2912
  time_since_restore: 0.31931090354919434
  time_this_iter_s: 0.10546493530273438
  time_total_s: 0.31931090354919434
  timestamp: 1638691640


Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_5c3cc7e3,RUNNING,127.0.0.1:9988,tanh,2,3,4,,,,,
easy_objective_5da68ece,RUNNING,127.0.0.1:8960,relu,-63.8792,3,17.361,,,,,
easy_objective_5c09f8a3,TERMINATED,127.0.0.1:2912,relu,2,3,1,8.53333,3,0.319311,2,-8.53333


Result for easy_objective_5c3cc7e3:
  date: 2021-12-05_21-07-21
  done: false
  experiment_id: 8ec1ab1b22b94bbc9b5481db62e2d841
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 127.0.0.1
  pid: 9988
  time_since_restore: 0.10060787200927734
  time_this_iter_s: 0.10060787200927734
  time_total_s: 0.10060787200927734
  timestamp: 1638691641
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5c3cc7e3
  
Result for easy_objective_5da68ece:
  date: 2021-12-05_21-07-22
  done: false
  experiment_id: 209ca2b75e604237b4ce1425c622ced5
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 3.6120771680213473
  neg_mean_loss: -3.6120771680213473
  node_ip: 127.0.0.1
  pid: 8960
  time_since_restore: 0.10155773162841797
  time_this_iter_s: 0.10155773162841797
  time_total_s: 0.10155773162841797
  timestamp: 1638691642
  timesteps_since_restore: 0
  training_iteration: 1
  tria

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_5c09f8a3,TERMINATED,127.0.0.1:2912,relu,2,3,1,8.53333,3,0.319311,2,-8.53333
easy_objective_5c3cc7e3,TERMINATED,127.0.0.1:9988,tanh,2,3,4,5.75556,3,0.544894,2,-5.75556
easy_objective_5da68ece,TERMINATED,127.0.0.1:8960,relu,-63.8792,3,17.361,-4.15189,3,0.322266,2,4.15189


2021-12-05 21:07:22,423	INFO tune.py:626 -- Total run time: 6.55 seconds (6.41 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 3, 'width': 17.36104198993424, 'height': -63.87922831978652, 'activation': 'relu'}


hyperopt_conditional_search_space_example

In [ ]:
"""This example demonstrates the usage of conditional search spaces with Tune.

It also checks that it is usable with a separate scheduler.

For an example of using a Tune search space, see
:doc:`/tune/examples/optuna_example`.
"""
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
from hyperopt import hp


def f_unpack_dict(dct):
    """
    Unpacks all sub-dictionaries in given dictionary recursively.
    There should be no duplicated keys across all nested
    subdictionaries, or some instances will be lost without warning

    Source: https://www.kaggle.com/fanvacoolt/tutorial-on-hyperopt

    Parameters:
    ----------------
    dct : dictionary to unpack

    Returns:
    ----------------
    : unpacked dictionary
    """

    res = {}
    for (k, v) in dct.items():
        if isinstance(v, dict):
            res = {**res, **f_unpack_dict(v)}
        else:
            res[k] = v

    return res


def evaluation_fn(step, width, height, mult=1):
    return (0.1 + width * step / 100)**(-1) + height * 0.1 * mult


def easy_objective(config_in):
    # Hyperparameters
    config = f_unpack_dict(config_in)
    width, height, mult = config["width"], config["height"], config.get(
        "mult", 1)
    print(config)

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height, mult)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)
        time.sleep(0.1)


config_space = {
    "activation": hp.choice("activation", [
        {
            "activation": "relu",
            "mult": hp.uniform("mult", 1, 2)
        },
        {
            "activation": "tanh"
        },
    ]),
    "width": hp.uniform("width", 0, 20),
    "height": hp.uniform("heright", -100, 100),
    "steps": 100
}


def run_hyperopt_tune(config_dict=config_space, smoke_test=False):
    algo = HyperOptSearch(space=config_dict, metric="mean_loss", mode="min")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=2 if smoke_test else 3,
    )

    print("Best hyperparameters found were: ", analysis.best_config)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
    if args.server_address is not None:
        ray.util.connect(args.server_address)
    else:
        ray.init(configure_logging=False)

    run_hyperopt_tune(smoke_test=args.smoke_test)

bayesopt_example

In [2]:
"""This example demonstrates the usage of BayesOpt with Ray Tune.

It also checks that it is usable with a separate scheduler.
"""
import time

from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)
        time.sleep(0.1)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()

    if args.server_address:
        import ray

        ray.init(f"ray://{args.server_address}")

    algo = BayesOptSearch(utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
    })
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    analysis = tune.run(
        easy_objective,
        name="my_exp",
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=2 if args.smoke_test else 3,
        config={
            "steps": 3,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100)
        })

    print("Best hyperparameters found were: ", analysis.best_config)

2021-12-05 21:04:14,087	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-05 21:04:18,866	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-05 21:04:18,878	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2021-12-05 21:04:18,880	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2021-12-05 21:04:19,000	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,height,width
easy_objective_f29cc20b,PENDING,,-25.092,19.0143


2021-12-05 21:04:19,634	ERROR syncer.py:111 -- Log sync requires rsync to be installed.


Result for easy_objective_f29cc20b:
  date: 2021-12-05_21-04-22
  done: false
  experiment_id: f34d7e2b0abd4f898c76da6ca51a154a
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 7.490802376947249
  neg_mean_loss: -7.490802376947249
  node_ip: 127.0.0.1
  pid: 4180
  time_since_restore: 0.0
  time_this_iter_s: 0.0
  time_total_s: 0.0
  timestamp: 1638691462
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f29cc20b
  
Result for easy_objective_f2fc44b3:
  date: 2021-12-05_21-04-22
  done: true
  experiment_id: ff579ef0d5da4c41a40d1c9b76b1ce67
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 14.639878836228101
  neg_mean_loss: -14.639878836228101
  node_ip: 127.0.0.1
  pid: 8688
  time_since_restore: 0.0
  time_this_iter_s: 0.0
  time_total_s: 0.0
  timestamp: 1638691462
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f2fc44b3
  
Result for easy_objective_f29cc20b:
  date: 2021-12-0

Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_f29cc20b,TERMINATED,127.0.0.1:4180,-25.092,19.0143,-0.427104,3,0.217759,2,0.427104
easy_objective_f2fc44b3,TERMINATED,127.0.0.1:8688,46.3988,11.9732,14.6399,1,0,0,-14.6399
easy_objective_f3228fde,TERMINATED,127.0.0.1:14608,-68.7963,3.11989,-0.721909,3,0.217899,2,0.721909


2021-12-05 21:04:23,445	INFO tune.py:626 -- Total run time: 4.59 seconds (4.32 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 3, 'width': 3.119890406724053, 'height': -68.79627191151269}


optuna_example

In [4]:
"""This example demonstrates the usage of Optuna with Ray Tune.

It also checks that it is usable with a separate scheduler.

For an example of using an Optuna define-by-run function, see
:doc:`/tune/examples/optuna_define_by_run_example`.
"""
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch


def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)
        time.sleep(0.1)


def run_optuna_tune(smoke_test=False):
    algo = OptunaSearch()
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=2 if smoke_test else 3,
        config={
            "steps": 3,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
            # This is an ignored parameter.
            "activation": tune.choice(["relu", "tanh"])
        })

    print("Best hyperparameters found were: ", analysis.best_config)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
#     if args.server_address is not None:
#         ray.init(f"ray://{args.server_address}")
#     else:
#         ray.init(configure_logging=False)

    run_optuna_tune(smoke_test=args.smoke_test)

[I 2021-12-05 21:05:31,783] A new study created in memory with name: optuna
2021-12-05 21:05:31,788	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,activation,height,width
easy_objective_1dfe5a56,PENDING,,tanh,-37.5212,8.10681


Result for easy_objective_1dfe5a56:
  date: 2021-12-05_21-05-35
  done: false
  experiment_id: 161fe7f3f7e341189212e3631bcdd18c
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 6.247879719956709
  neg_mean_loss: -6.247879719956709
  node_ip: 127.0.0.1
  pid: 3852
  time_since_restore: 0.0
  time_this_iter_s: 0.0
  time_total_s: 0.0
  timestamp: 1638691535
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1dfe5a56
  
Result for easy_objective_1dfe5a56:
  date: 2021-12-05_21-05-35
  done: true
  experiment_id: 161fe7f3f7e341189212e3631bcdd18c
  experiment_tag: 1_activation=tanh,height=-37.521,steps=3,width=8.1068
  hostname: LAPTOP-5F3V9FNM
  iterations: 2
  iterations_since_restore: 3
  mean_loss: 0.06268917159503307
  neg_mean_loss: -0.06268917159503307
  node_ip: 127.0.0.1
  pid: 3852
  time_since_restore: 0.21678614616394043
  time_this_iter_s: 0.10546588897705078
  time_total_s: 0.21678614616394043
  timestamp: 1638691535
  timest

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_1e37b7ee,RUNNING,127.0.0.1:15048,tanh,24.9046,9.25847,,,,,
easy_objective_1fc246b4,RUNNING,127.0.0.1:10404,relu,-10.03,18.9729,,,,,
easy_objective_1dfe5a56,TERMINATED,127.0.0.1:3852,tanh,-37.5212,8.10681,0.0626892,3,0.216786,2,-0.0626892


Result for easy_objective_1fc246b4:
  date: 2021-12-05_21-05-37
  done: true
  experiment_id: d8b257da0727462e89ea94f60a9a56db
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 8.996997315053644
  neg_mean_loss: -8.996997315053644
  node_ip: 127.0.0.1
  pid: 10404
  time_since_restore: 0.0
  time_this_iter_s: 0.0
  time_total_s: 0.0
  timestamp: 1638691537
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1fc246b4
  
Result for easy_objective_1e37b7ee:
  date: 2021-12-05_21-05-37
  done: true
  experiment_id: 3ef7ae8f10414ef8aed8493ca2bfb7d7
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 12.490457877873144
  neg_mean_loss: -12.490457877873144
  node_ip: 127.0.0.1
  pid: 15048
  time_since_restore: 0.013674736022949219
  time_this_iter_s: 0.013674736022949219
  time_total_s: 0.013674736022949219
  timestamp: 1638691537
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1e37b7ee
  


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_1dfe5a56,TERMINATED,127.0.0.1:3852,tanh,-37.5212,8.10681,0.0626892,3,0.216786,2,-0.0626892
easy_objective_1e37b7ee,TERMINATED,127.0.0.1:15048,tanh,24.9046,9.25847,12.4905,1,0.0136747,0,-12.4905
easy_objective_1fc246b4,TERMINATED,127.0.0.1:10404,relu,-10.03,18.9729,8.997,1,0,0,-8.997


2021-12-05 21:05:38,081	INFO tune.py:626 -- Total run time: 6.31 seconds (6.15 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 3, 'width': 8.106814543129218, 'height': -37.52120280043292, 'activation': 'tanh'}


optuna_define_by_run_example

In [5]:
"""This example demonstrates the usage of Optuna define-by-run with Ray Tune.

It also checks that it is usable with a separate scheduler.

For an example of using a Tune search space, see
:doc:`/tune/examples/optuna_example`.
"""
import time
from typing import Dict, Optional, Any

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch


def evaluation_fn(step, width, height, mult=1):
    return (0.1 + width * step / 100)**(-1) + height * 0.1 * mult


def easy_objective(config):
    # Hyperparameters
    width, height, mult = config["width"], config["height"], config.get(
        "mult", 1)
    print(config)

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height, mult)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)
        time.sleep(0.1)


def define_by_run_func(trial) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``tune.run`` (in this example, that's
    ``easy_objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
/tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """
    # This param is not used in the objective function.
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    trial.suggest_float("width", 0, 20)
    trial.suggest_float("height", -100, 100)

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("mult", 1, 2)

    # Return all constants in a dictionary.
    return {"steps": 100}


def run_optuna_tune(smoke_test=False):
    algo = OptunaSearch(
        space=define_by_run_func, metric="mean_loss", mode="min")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=2 if smoke_test else 3,
    )

    print("Best hyperparameters found were: ", analysis.best_config)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
#     if args.server_address is not None:
#         ray.util.connect(args.server_address)
#     else:
#         ray.init(configure_logging=False)

    run_optuna_tune(smoke_test=args.smoke_test)

[I 2021-12-05 21:06:38,260] A new study created in memory with name: optuna
2021-12-05 21:06:38,279	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,activation,height,steps,width
easy_objective_459ffa9e,PENDING,,tanh,-61.2795,100,5.57646


 pid=7212) {'activation': 'tanh', 'width': 5.576461330546616, 'height': -61.27945918068119, 'steps': 100}


Trial name,status,loc,activation,height,steps,width
easy_objective_459ffa9e,RUNNING,127.0.0.1:7212,tanh,-61.2795,100,5.57646
easy_objective_474914e4,RUNNING,127.0.0.1:11900,tanh,80.4224,100,9.64474
easy_objective_476e08dd,PENDING,,tanh,84.2352,100,10.2108


Result for easy_objective_474914e4:
  date: 2021-12-05_21-06-44
  done: false
  experiment_id: bd86de995ac04d83a1e3012122e17b5a
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 18.04224148033588
  neg_mean_loss: -18.04224148033588
  node_ip: 127.0.0.1
  pid: 11900
  time_since_restore: 0.0
  time_this_iter_s: 0.0
  time_total_s: 0.0
  timestamp: 1638691604
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 474914e4
  
Result for easy_objective_459ffa9e:
  date: 2021-12-05_21-06-44
  done: false
  experiment_id: 508c1df676b74601b0dc836226ff8559
  hostname: LAPTOP-5F3V9FNM
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 3.872054081931881
  neg_mean_loss: -3.872054081931881
  node_ip: 127.0.0.1
  pid: 7212
  time_since_restore: 0.0009779930114746094
  time_this_iter_s: 0.0009779930114746094
  time_total_s: 0.0009779930114746094
  timestamp: 1638691604
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 459ffa9e
  

...

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_459ffa9e,RUNNING,127.0.0.1:7212,tanh,-61.2795,100,5.57646,-5.92133,86,9.47596,85,5.92133
easy_objective_474914e4,TERMINATED,127.0.0.1:11900,tanh,80.4224,100,9.64474,10.6107,4,0.399392,3,-10.6107
easy_objective_476e08dd,TERMINATED,127.0.0.1:2892,tanh,84.2352,100,10.2108,18.4235,1,0,0,-18.4235


Result for easy_objective_459ffa9e:
  date: 2021-12-05_21-06-54
  done: false
  experiment_id: 508c1df676b74601b0dc836226ff8559
  hostname: LAPTOP-5F3V9FNM
  iterations: 94
  iterations_since_restore: 95
  mean_loss: -5.940745683583656
  neg_mean_loss: 5.940745683583656
  node_ip: 127.0.0.1
  pid: 7212
  time_since_restore: 10.43297529220581
  time_this_iter_s: 0.10453510284423828
  time_total_s: 10.43297529220581
  timestamp: 1638691614
  timesteps_since_restore: 0
  training_iteration: 95
  trial_id: 459ffa9e
  
Result for easy_objective_459ffa9e:
  date: 2021-12-05_21-06-55
  done: true
  experiment_id: 508c1df676b74601b0dc836226ff8559
  hostname: LAPTOP-5F3V9FNM
  iterations: 99
  iterations_since_restore: 100
  mean_loss: -5.950032031177093
  neg_mean_loss: 5.950032031177093
  node_ip: 127.0.0.1
  pid: 7212
  time_since_restore: 10.96609616279602
  time_this_iter_s: 0.10542464256286621
  time_total_s: 10.96609616279602
  timestamp: 1638691615
  timesteps_since_restore: 0
  trainin

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_459ffa9e,TERMINATED,127.0.0.1:7212,tanh,-61.2795,100,5.57646,-5.95003,100,10.9661,99,5.95003
easy_objective_474914e4,TERMINATED,127.0.0.1:11900,tanh,80.4224,100,9.64474,10.6107,4,0.399392,3,-10.6107
easy_objective_476e08dd,TERMINATED,127.0.0.1:2892,tanh,84.2352,100,10.2108,18.4235,1,0,0,-18.4235


2021-12-05 21:06:55,209	INFO tune.py:626 -- Total run time: 16.95 seconds (16.80 seconds for the tuning loop).


Best hyperparameters found were:  {'activation': 'tanh', 'width': 5.576461330546616, 'height': -61.27945918068119, 'steps': 100}
